In [2]:
import pandas as pd
import numpy as np
import sklearn as skl
import seaborn as sns

In [ ]:
df = pd.read_excel('apartment_deals_price.xlsx', sheet_name='selected_data')

In [ ]:
names=['city_1', 'city_2', 'city_3', 'address', 'size', 'floor', 'built_year', 'price']
df = df[names]

In [ ]:
df['new_address'] = df['city_2'] + " " + df['address']

In [ ]:
from geopy import Nominatim
from geopy import distance

import requests
from urllib.parse import urlparse

dist_dict = dict()
lati_dict = dict()
long_dict = dict()

no_addr = []

locator = Nominatim(user_agent="myGeocoder")
city_hall_coor = (locator.geocode("세종대로 110").latitude, locator.geocode("세종대로 110").longitude)

for idx, addr in enumerate(df['new_address'].unique()):
	try:
		url = 'https://dapi.kakao.com/v2/local/search/address.json?&query=' + addr
		result = requests.get(urlparse(url).geturl(), headers={'Authorization': 'KakaoAK b835f8f34eed2ee673250126a3f100f4'}).json()
		match_first = result['documents'][0]['address']

		# location = locator.geocode(addr)
		# lati[addr] = location.latitude
		# long[addr] = location.longitude
		lati_dict[addr] = float(match_first['y'])
		long_dict[addr] = float(match_first['x'])
		dist_dict[addr] = distance.distance(city_hall_coor, (lati_dict[addr], long_dict[addr])).km

		print('pct: {}%, addr: {} => (lat: {}, long: {}, dist: {})'.format((idx+1)/5794*100, addr, lati_dict[addr], long_dict[addr], dist_dict[addr]))
	except:
		print(addr, 'could not find its coordinates')
		no_addr.append(addr)

In [ ]:
new_addr = {
	'강남구 영동대로128길 15':'37.5199625,127.0573796',
	'강동구 천중로 264':'37.5396021,127.1472591',
	'강동구 진황도로 18':'37.5393977,127.1267621',
	'강동구 진황도로 12':'37.5386759,127.1261437',
	'강서구 화곡로21길 71-1':'37.5403939,126.8330997',
	'강서구 곰달래로18길 33-1':'37.5287512,126.8400154',
	'관악구  27':'37.4842408,126.9354367',
	'관악구 구암길 95':'37.4915053,126.9445855',
	'광진구 뚝섬로 45':'37.5336664,127.0755005',
	'구로구 구로중앙로18길 11-1':'37.4950835,126.8878291',
	'금천구 시흥대로 291-1':'37.4611632,126.8927868',
	'동대문 한천로24길 74-11':'37.5681599,127.0661988',
	'동작구 사당로23길 4':'37.486995,126.9678221',
	'동작구  27':'37.485685,126.9606768',
	'마포구 효창원로93길 18':'37.5484817,126.9600567',
	'서초구 강남대로39길 15-1':'37.4858948,127.0313779',
	'서초구 나루터로4길 49':'37.5105427,127.0118349',
	'서초구 나루터로4길 28':'37.5115117,127.0101548',
	'서초구 잠원로4길 34-11':'37.5117162,127.0089611',
	'서초구 신반포로33길 30':'37.5112721,127.0082226',
	'서초구 나루터로4길 39':'37.5125168,127.011931',
	'서초구 신반포로 267':'37.5091089,127.0114771',
	'성북구  37':'37.5840776,127.0187608',
	'송파구 올림픽로 399':'37.5183239,127.1085036',
	'용산구 한강대로62길 45-1':'37.5328297,126.971493',
	'용산구 유엔빌리지길 80-1':'37.5322288,127.0091033',
	'은평구 서오릉로2길 12-1':'37.6021139,126.9256897',
	'은평구 통일로65길 1-1':'37.6090074,126.9284392',
	'은평구 서오릉로 80-1':'37.6074525,126.919948',
	'은평구 통일로65길 3-1':'37.608937,126.9283202',
	'은평구 연서로28길 12-1':'37.6172734,126.9196284',
	'은평구 연서로28길 8-1':'37.6177852,126.918537',
	'은평구 서오릉로 119-1':'37.6099049,126.9166778',
	'은평구 서오릉로 107-1':'37.6092379,126.9175358',
	'은평구 은평로 116-1':'37.5999023,126.9185145',
	'종로구 평창문화로 171':'37.6109493,126.9764674',
	'종로구 세검정로 384-13':'37.6075659,126.9711736',
	'중구 서소문로 89-1':'37.5625543,126.9686235',
	'중랑구 봉화산로1길 16':'37.6023589,127.0731882',
}

for addr in new_addr.keys():
	lat, long = new_addr[addr].split(',')
	lati_dict[addr] = float(lat)
	long_dict[addr] = float(long)
	print(lati_dict[addr], long_dict[addr])

In [ ]:
long_df = pd.DataFrame.from_dict(long_dict, 'index', columns=['longitude'])
lati_df = pd.DataFrame.from_dict(lati_dict, 'index', columns=['latitude'])
dist_df = pd.DataFrame.from_dict(dist_dict, 'index', columns=['distance'])

lat_lon_dist_df = df.join(lati_df, on="new_address").join(long_df, on="new_address").join(dist_df, on="new_address")
lat_lon_dist_df.to_csv('result_lat_lon_dist.csv', index=False)

In [ ]:
dist_df = pd.read_csv('result_lat_lon_dist.csv')
dist_df.groupby('city_2').mean()

In [2]:
from geopy import Nominatim

locator = Nominatim(user_agent="myGeocoder")
city_hall_coor = (locator.geocode("세종대로 110").latitude, locator.geocode("세종대로 110").longitude)
city_hall_coor

(37.56635025, 126.97830900276097)

# Graph #

In [ ]:
import matplotlib.pyplot as plt

# sns.regplot(x='size', y='price', data=dist_df)
sns.regplot(x='distance', y='price', data=dist_df)

# Regression #

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score

y_target = dist_df['price']
# X_data = dist_df.drop(['price', 'area', 'building_number', 'apartment_name', 'address'], axis=1, inplace=False)
X_data = dist_df[['size']]

X_train , X_test , y_train , y_test = train_test_split(X_data, y_target, test_size=0.3, random_state=156)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_test, y_preds)))

In [ ]:
coeff = pd.Series(data=np.round(lr.coef_, 2), index=X_data.columns)
coeff.sort_values(ascending=False)

In [ ]:
import math

coeff = pd.Series(data=np.round(lr.coef_, 2), index=X_data.columns)
coeff.sort_values(ascending=False)

coeff.sort_values(ascending=False, inplace=True)
res = 'PRICE = '
for idx, item in enumerate(coeff.items()):
  name, value = item
  if idx == 0:
    op = ''
  else:
    if value > 0:
      op = ' + '
    else:
      op = ' - '
  res += op + '*'.join([str(math.fabs(value)), name.upper()])

if lr.intercept_ > 0:
  res += " + " + str(math.fabs(lr.intercept_))
else:
  res += " - " + str(math.fabs(lr.intercept_))

# print()

print('[Linear regression]')
print(res)

print()

print('[Sorted covariance of all variables in terms of price]')
dist_df.cov()['price'][dist_df.cov()['price'].abs().sort_values(ascending=False).keys()]